<a href="https://colab.research.google.com/github/DonnaVakalis/Urban.dat/blob/master/CensusDat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyze Toronto's Rooftop Solar Capacity

 

## Table of Contents
- [Introduction](#intro)
- [Part I - Setup ](#partone)
- [Part II - Calculations from scratch ](#parttwo)
- [Part III - Graphs](#partthree)


<a id='intro'></a>
### Introduction
 


<a id='partone'></a>
### Part I - Setup

In [1]:
# import libraries

!pip install statsmodels

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
from google.colab import drive
import statsmodels as sm

%matplotlib inline
 

In [ ]:
# Download 2016 Census data from StatsCan

req = requests.get(csv_url)
url_content = req.content
csv_file = open('downloaded.csv', 'wb')

 https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/dt-td/CompDataDownload.cfm?LANG=E&PID=111332&OFT=CSV

In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Load csv files
base_dir =  "/content/gdrive/My Drive/Colab Notebooks/SSG/Census/"

# all residential buildings
file = base_dir + "xyz.csv"
df1 = pd.read_csv(file)

# load all commercial buildings
file = base_dir + "wer.csv"
df2= pd.read_csv(file) 

Cleaning Notes:
- Pretty clean (not surprising given it was output from model)
- Based on the magnitude of the floor areas 'bsmtArea_sf' column , these are not individual buildings and for now don't know the underlying count of buildings that have been aggregated into the number in ... e.g., so can't subtract the perimeter or check for minimum indivisual size as per EIE method


<a id='parttwo></a>
### Part II -  

Assumptions:
-  
Other:
- Use an efficiency performance factor of 75% to account for losses (e.g., climatic factors, inverter, conversion losses...)


In [ ]:
# Set Assumptions as named variables

PART IIA - SSG RESIDENTIAL

<a id='partthree'></a>
### Part III -  
 

 Assumptions:
 

Part III A:  

Part III B:  
